In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from scipy.special import logit
from scipy.stats import norm
from functions import  feature_engineering, train_test_split, one_hot
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

In [2]:
data = pd.read_csv("/Users/victorialokteva/Downloads/data.csv")
data = data.drop(columns=["oaid_hash", 'impressions', 'campaign_clicks'])


In [3]:
data.head()

,date_time,zone_id,banner_id,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,clicks
0,2021-09-27 00:01:30.000000,0,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1
1,2021-09-26 22:54:49.000000,1,1,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1
2,2021-09-26 23:57:20.000000,2,2,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1
3,2021-09-27 00:04:30.000000,3,3,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1
4,2021-09-27 00:06:21.000000,4,4,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1


In [3]:
data = feature_engineering(data)


In [5]:
data.head()

,date_time,zone_id,banner_id,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,clicks,time,date,time_of_day,is_weekend,month,date_copy
0,2021-09-27 00:01:30,0,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1,00:01:30,2021-09-27,0.00,0,0,2021-09-27
1,2021-09-26 22:54:49,1,1,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,22:54:49,2021-09-26,0.25,1,0,2021-09-26
2,2021-09-26 23:57:20,2,2,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,23:57:20,2021-09-26,0.25,1,0,2021-09-26
3,2021-09-27 00:04:30,3,3,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,00:04:30,2021-09-27,0.00,0,0,2021-09-27
4,2021-09-27 00:06:21,4,4,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,00:06:21,2021-09-27,0.00,0,0,2021-09-27


In [4]:
X_train, Y_train, X_test0, X_test1, Y_test, coefs = train_test_split(data)
X_train.to_csv("X_train.csv")
Y_train.to_csv("Y_train.csv")
X_test0.to_csv("X_test0.csv")
X_test1.to_csv("X_test1.csv")
Y_test.to_csv("Y_test.csv")
coefs.to_csv("coefs.csv")

In [2]:
X_train, Y_train = pd.read_csv("X_train.csv"), pd.read_csv("Y_train.csv")
X_test0, X_test1 = pd.read_csv("X_test0.csv"), pd.read_csv("X_test1.csv")
Y_test, coefs = pd.read_csv("Y_test.csv"), pd.read_csv("coefs.csv")

In [ ]:
X_train = one_hot(X_train)
X_train.to_csv("X_trainOH.csv")

In [ ]:
X_test0 = one_hot(X_test0)
X_test0.to_csv("X_test0OH.csv")

In [1]:
X_test1 = one_hot(X_test1)
X_test1.to_csv("X_test1OH.csv")

NameError: name 'one_hot' is not defined

In [ ]:
X_train, X_test0, X_test1 = one_hot(X_train), one_hot(X_test0), one_hot(X_test1 )

In [ ]:
X_train.to_csv("X_train.csv")
Y_train.to_csv("Y_train.csv")
X_test0.to_csv("X_test0.csv")
X_test1.to_csv("X_test1.csv")
Y_test.to_csv("Y_test.csv")
coefs.to_csv("coefs.csv")

In [1]:
# Модель из 1 дз
train = pd.concat([X_train, Y_train])
model = LogisticRegression(penalty='l2', solver='liblinear', C=0.1)
model.fit(X_train, Y_train)

NameError: name 'pd' is not defined

In [1]:
y_pred0 = model.predict(X_test0)
y_pred1 = model.predict(X_test1)
coefs['coeff_sum0_new'] = logit(model.predict_proba(X_test0)[:, 1])
coefs['coeff_sum1_new'] = logit(model.predict_proba(X_test1)[:, 1])

NameError: name 'model' is not defined

In [ ]:
coefs['pi_0'] = norm.sf((coefs['coeff_sum1'] -
                          coefs['coeff_sum0']) / np.sqrt(coefs['g0'] ** 2 + coefs['g1'] ** 2))
coefs['pi_1'] = norm.sf((coefs['coeff_sum1_new'] -
                          coefs['coeff_sum0_new']) / np.sqrt(coefs['g0'] ** 2 + coefs['g1'] ** 2))

In [ ]:
lam = 10
cips = np.sum(Y_test * np.clip(coefs['pi_1'] / coefs['pi_0'], a_min=None, a_max=lam)) / len(Y_test)
print("Cips: ", cips)